<a href="https://colab.research.google.com/github/angomezh1/Curso2/blob/master/Clase_extra_guardado_de_modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p><img alt="Colaboratory logo" height="140px" src="https://upload.wikimedia.org/wikipedia/commons/archive/f/fb/20161010213812%21Escudo-UdeA.svg" align="left" hspace="10px" vspace="0px"></p>

<h1> Diplomado de Análisis de datos y Machine Learning en Python</h1>


El presente diplomado hace parte del centro de Big Data de la facultad de ciencias exactas y naturales (FCEN) de la Universidad de Antioquia.

In [0]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
!pip install h5py

In [0]:
!wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv

--2019-12-12 23:42:57--  https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23278 (23K) [text/plain]
Saving to: ‘pima-indians-diabetes.data.csv’

pima-indians-diabet 100%[===================>]  22.73K  --.-KB/s    in 0.007s  

2019-12-12 23:42:58 (3.29 MB/s) - ‘pima-indians-diabetes.data.csv’ saved [23278/23278]



Cuando estamos creando modelos de deep learning  estámos calculando una serie de pesos $w$ que ajusten lo mejor posible (dependiendo de nuestra arquitectura, funciones de perdida y optimizadores) una serie de datos.

Dichos modelos serán usados de manera posterior como parte de algún proceso más complejo. Si tuvieramos que entrenar un modelo cada que lo deseamos usar o cada que reiniciamos la aplicación sería intratable; los modelos reales pueden tomar horas, incluso semanas o meses en ser entrenados, por tanto necesitamos una forma de guardar nuestros pesos y la información de nuestra arquitectura para ser usada de manera posterior.

# Guardar el modelo en un JSON

In [0]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import model_from_json
import numpy
import os

numpy.random.seed(7)

In [0]:
!ls

pima-indians-diabetes.data.csv	sample_data


In [0]:
dataset = numpy.loadtxt("./pima-indians-diabetes.data.csv", delimiter=",")

X = dataset[:,0:8]
Y = dataset[:,8]

In [0]:
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12)                108       
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 104       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
Total params: 221
Trainable params: 221
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, Y, epochs=150, batch_size=10, verbose=0)
scores = model.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

accuracy: 76.82%


In [0]:
#guardamos la arquitectura en un archivo json
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
#guardamos los pesos en un archivo h5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
!ls

model.h5  model.json  pima-indians-diabetes.data.csv  sample_data


In [0]:
#cargamos la arquitectura
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
#cargamos los pesos
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [0]:
#recompilamos el modelo
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

accuracy: 76.82%


In [0]:
!cat model.json

{"class_name": "Sequential", "config": {"name": "sequential", "layers": [{"class_name": "Dense", "config": {"name": "dense", "trainable": true, "batch_input_shape": [null, 8], "dtype": "float32", "units": 12, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dense", "config": {"name": "dense_1", "trainable": true, "dtype": "float32", "units": 8, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dense", "config": {"name": "dense_

Como vemos, hemos guardado por separado los pesos de los datos que definen la arquitectura.
En éste caso es necesario recompilar, y para ello saber cuales fueron las funciones de perdida, los optimizadores y las metricas usadas en el modelo al ser guardado en disco.

# Guardar el modelo en un YAML

También nos es posible guardar la arquitectura en un archivo YAML, el cual es parecido a un archivo JSON pero separado por lineas.

In [0]:
!pip install PyYAML

In [0]:
from tensorflow.keras.models import model_from_yaml

In [0]:
model2 = Sequential()
model2.add(Dense(12, input_dim=8, activation='relu'))
model2.add(Dense(8, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))

In [0]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 12)                108       
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 104       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 9         
Total params: 221
Trainable params: 221
Non-trainable params: 0
_________________________________________________________________


In [0]:
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.fit(X, Y, epochs=150, batch_size=10, verbose=0)

scores = model2.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (model2.metrics_names[1], scores[1]*100))

accuracy: 76.69%


In [0]:
#guardamos la arquitectura en un archivo YAML
model_yaml = model2.to_yaml()
with open("model2.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
#guardamos los pesos en un archivo h5
model2.save_weights("model2.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
!ls

model2.h5    model.h5	 pima-indians-diabetes.data.csv
model2.yaml  model.json  sample_data


In [0]:
#cargamos la arquitectura
yaml_file = open('model2.yaml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
loaded_model = model_from_yaml(loaded_model_yaml)
#cargamos los pesos
loaded_model.load_weights("model2.h5")
print("Loaded model from disk")

Loaded model from disk


In [0]:
#recompilamos el modelo
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

accuracy: 76.69%


In [0]:
from google.colab import files

In [0]:
files.download('model2.h5')

In [0]:
!cat model2.yaml

backend: tensorflow
class_name: Sequential
config:
  layers:
  - class_name: Dense
    config:
      activation: relu
      activity_regularizer: null
      batch_input_shape: !!python/tuple [null, 8]
      bias_constraint: null
      bias_initializer:
        class_name: Zeros
        config: {}
      bias_regularizer: null
      dtype: float32
      kernel_constraint: null
      kernel_initializer:
        class_name: GlorotUniform
        config: {seed: null}
      kernel_regularizer: null
      name: dense_3
      trainable: true
      units: 12
      use_bias: true
  - class_name: Dense
    config:
      activation: relu
      activity_regularizer: null
      bias_constraint: null
      bias_initializer:
        class_name: Zeros
        config: {}
      bias_regularizer: null
      dtype: float32
      kernel_constraint: null
      kernel_initializer:
        class_name: GlorotUniform
        config: {seed: null}
      kernel_regularizer: null
      name: dense_4
      trainable:

De igual forma nos es necesario conocer los detalles del modelado (función de perdida, optimizador y metricas usadas) para poder recompilar el modelo, pero hecho ésto el modelo es exactamente el mismo que el que se tenía al momento de guardado.

# Guardar el modelo y los pesos en un mismo archivo

Si deseamos guardar toda la información del modelado en el mismo archivo (incluidas las funciones de perdida, optimización y metrica), como suele suceder, podemos hacerlo en un solo archivo h5.

In [0]:
model3 = Sequential()
model3.add(Dense(12, input_dim=8, activation='relu'))
model3.add(Dense(8, activation='relu'))
model3.add(Dense(1, activation='sigmoid'))

In [0]:
model3.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 12)                108       
_________________________________________________________________
dense_7 (Dense)              (None, 8)                 104       
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 9         
Total params: 221
Trainable params: 221
Non-trainable params: 0
_________________________________________________________________


In [0]:
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model3.fit(X, Y, epochs=150, batch_size=10, verbose=0)
scores = model3.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (model3.metrics_names[1], scores[1]*100))

accuracy: 76.56%


In [0]:
#guardamos todos los datos del modelado juntos
model3.save("model3.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
!ls

model2.h5    model3.h5	model.json			sample_data
model2.yaml  model.h5	pima-indians-diabetes.data.csv


# Cargar un modelo completo desde el disco

In [0]:
from tensorflow.keras.models import load_model
model4 = load_model('model3.h5')
model4.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 12)                108       
_________________________________________________________________
dense_7 (Dense)              (None, 8)                 104       
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 9         
Total params: 221
Trainable params: 221
Non-trainable params: 0
_________________________________________________________________


In [0]:
score = model4.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (model4.metrics_names[1], score[1]*100))

accuracy: 76.56%


Note que en éste caso no es necesario el recompilado del modelo, pues el archivo h5 contiene toda la información necesaria al ser cargado en memoria.

#Continuar el entrenamiento de manera posterior

Al ser tan costoso el entrenamiento del modelo, o incluso al ser posible hacer transfer learning con finetuning, nos es posible cargar un modelo desde disco (con cualquiera de los métodos anteriores) y continuar entrenandolo con nuevos datos o por más épocas.

In [0]:
model4.fit(X, Y, epochs=10, batch_size=10)

Train on 768 samples
Epoch 1/10
768/768 [==============================] - 0s 519us/sample - loss: 0.5034 - accuracy: 0.7695
Epoch 2/10
768/768 [==============================] - 0s 259us/sample - loss: 0.4910 - accuracy: 0.7695
Epoch 3/10
768/768 [==============================] - 0s 269us/sample - loss: 0.4923 - accuracy: 0.7630
Epoch 4/10
768/768 [==============================] - 0s 277us/sample - loss: 0.4870 - accuracy: 0.7773
Epoch 5/10
768/768 [==============================] - 0s 262us/sample - loss: 0.4925 - accuracy: 0.7630
Epoch 6/10
768/768 [==============================] - 0s 265us/sample - loss: 0.4873 - accuracy: 0.7591
Epoch 7/10
768/768 [==============================] - 0s 293us/sample - loss: 0.4908 - accuracy: 0.7839
Epoch 8/10
768/768 [==============================] - 0s 287us/sample - loss: 0.4814 - accuracy: 0.7812
Epoch 9/10
768/768 [==============================] - 0s 261us/sample - loss: 0.4870 - accuracy: 0.7630
Epoch 10/10
768/768 [======================

#Guardar el modelo cada cierto tiempo (epoch)

Nos es posible incluso guardar modelos que están siendo entrenados en etápas intermedias, ésto para probar su evolución o con intereses investigativos.

Ésto lo podemos hacer con los `callbacks`, al igual que hacemos el `earlystoping`.

In [0]:
model5 = Sequential()
model5.add(Dense(12, input_dim=8, activation='relu'))
model5.add(Dense(8, activation='relu'))
model5.add(Dense(1, activation='sigmoid'))
model5.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 12)                108       
_________________________________________________________________
dense_10 (Dense)             (None, 8)                 104       
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 9         
Total params: 221
Trainable params: 221
Non-trainable params: 0
_________________________________________________________________


In [0]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

model_checkpoint = ModelCheckpoint('modelo5-{epoch:02d}.h5', period=2,save_weights_only=False)
early_stopping = EarlyStopping(patience=3, restore_best_weights=True)

In [0]:
!ls

model2.h5    model3.h5	model.json			sample_data
model2.yaml  model.h5	pima-indians-diabetes.data.csv


In [0]:
model5.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model5.fit(X, Y, epochs=10, batch_size=10, callbacks=[model_checkpoint, early_stopping]) 
#note que podemos incluir en el arreglo de callbacks cuantos funciones de callback queramos

Train on 768 samples
Epoch 1/10
768/768 [==============================] - 1s 899us/sample - loss: 5.8826 - accuracy: 0.4740
Epoch 2/10
768/768 [==============================] - 0s 305us/sample - loss: 2.3641 - accuracy: 0.4362
Epoch 3/10
768/768 [==============================] - 0s 265us/sample - loss: 1.2968 - accuracy: 0.4518
Epoch 4/10
768/768 [==============================] - 0s 288us/sample - loss: 0.9624 - accuracy: 0.4974
Epoch 5/10
768/768 [==============================] - 0s 294us/sample - loss: 0.8545 - accuracy: 0.5742
Epoch 6/10
768/768 [==============================] - 0s 282us/sample - loss: 0.7608 - accuracy: 0.6159
Epoch 7/10
768/768 [==============================] - 0s 268us/sample - loss: 0.7107 - accuracy: 0.6484
Epoch 8/10
768/768 [==============================] - 0s 284us/sample - loss: 0.7011 - accuracy: 0.6315
Epoch 9/10
768/768 [==============================] - 0s 281us/sample - loss: 0.6712 - accuracy: 0.6549
Epoch 10/10
768/768 [======================

In [0]:
!ls

model2.h5    model.h5	    modelo5-04.h5  modelo5-10.h5
model2.yaml  model.json     modelo5-06.h5  pima-indians-diabetes.data.csv
model3.h5    modelo5-02.h5  modelo5-08.h5  sample_data
